<a href="https://colab.research.google.com/github/SamuelPassamani/XCam/blob/main/xcam-colab/XCam%20GIFs/Conversor%20M3U8-GIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Monta o Google Drive no ambiente do Colab
drive.mount('/content/drive')

print("✅ Google Drive montado com sucesso!")

**Reasoning**:
The error "name 'PRIMARY_API_URL' is not defined" indicates that the global configuration variables (like `PRIMARY_API_URL`, `SECONDARY_API_URL_TEMPLATE`, `DRIVE_FOLDER_PATH`, `LOG_FILE_NAME`, `TARGET_GIF_SIZE_MB`, `SAFETY_FACTOR`, `MAX_CLIP_SECONDS_TO_ANALYZE`, `MIN_GIF_DURATION`, `GIF_FPS`, `GIF_WIDTH`, `WATERMARK_PATH`, `WATERMARK_WIDTH`) which were defined in the second cell are not accessible in the current execution context. To resolve this, I need to include the definition of these configuration variables in the current code block or ensure they are loaded from a source that is accessible. Since they were defined as simple variables, redefining them in this cell is the most straightforward approach to make them available to the code that uses them. I will copy the configuration block from the second cell.



In [ ]:
import requests
import subprocess
import os
import json
import multiprocessing # Import multiprocessing module
from functools import partial # Import partial for passing fixed arguments to pool map

# ==============================================================================
# === CONFIGURAÇÕES GLOBAIS ===
# ==============================================================================

# --- Configurações da API ---
API_KEY = "99090882"
PRIMARY_API_URL = f"https://api.xcam.gay/?limit=1500&key={API_KEY}"
SECONDARY_API_URL_TEMPLATE = f"https://api.xcam.gay/?user={{username}}&key={API_KEY}"

# --- Configurações de Saída (Google Drive) ---
DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Projetos/XCam/Conteúdo Social/XCam Social Mídias/XCam GIFs'
LOG_FILE_NAME = "_usuarios_processados.json" # Alterado para JSON para facilitar o gerenciamento de status

# --- Configurações de Download e Conversão (AJUSTADO PARA CONTROLE DE TAMANHO) ---
# Alvo de tamanho máximo para o GIF final em Megabytes (MB).
TARGET_GIF_SIZE_MB = 14.14

# Fator de segurança para garantir que o GIF final fique ABAIXO do alvo.
# 0.95 = 95% do alvo. Aumente se os GIFs ainda estiverem passando do limite.
SAFETY_FACTOR = 0.95

# Duração máxima do clipe de vídeo a ser baixado para análise.
# O script usará este clipe para calcular a duração ideal do GIF. 30 segundos é um bom valor.
MAX_CLIP_SECONDS_TO_ANALYZE = 30

# Duração mínima para um GIF ser considerado útil (em segundos).
MIN_GIF_DURATION = 2.0

# Qualidade do GIF (mantida dos scripts anteriores)
GIF_FPS = 15
GIF_WIDTH = 640

# --- Configurações de Marca D'água ---
# Caminho para o arquivo da marca d'água (PNG, JPG, etc.).
# Especifique o caminho completo no seu Google Drive.
# Exemplo: '/content/drive/MyDrive/MinhaMarcaDagua.png'
WATERMARK_PATH = '/content/drive/MyDrive/Projetos/XCam/Conteúdo Social/XCam Social Mídias/logo.svg' # <--- COLOQUE O CAMINHO DA SUA MARCA D'ÁGUA AQUI!

# Largura desejada para a marca d'água em pixels.
WATERMARK_WIDTH = 115 # <--- AJUSTE AQUI A LARGURA DA SUA MARCA D'ÁGUA

# --- Configurações de Paralelismo ---
# Número de processos de worker a serem usados. None usa o número de núcleos da CPU.
NUM_WORKERS = 15 # Ajuste para um número específico se necessário, ex: 4


# ==============================================================================
# === FUNÇÕES AUXILIARES PARA O LOG ===
# ==============================================================================

def read_log(log_path):
    """Lê o arquivo de log JSON."""
    if not os.path.exists(log_path):
        return {}
    try:
        with open(log_path, 'r') as f:
            # Handle empty file case
            content = f.read()
            if not content:
                return {}
            return json.loads(content)
    except json.JSONDecodeError:
        print(f"⚠️ Atenção: Arquivo de log ({LOG_FILE_NAME}) está corrompido ou vazio. Criando um novo.")
        return {}
    except Exception as e:
        print(f"❌ Erro ao ler arquivo de log: {e}")
        return {}

def write_log(log_path, log_data):
    """Escreve os dados no arquivo de log JSON."""
    try:
        with open(log_path, 'w') as f:
            json.dump(log_data, f, indent=4)
    except Exception as e:
        print(f"❌ Erro ao escrever no arquivo de log: {e}")

# ==============================================================================
# === FUNÇÃO DE PROCESSAMENTO POR USUÁRIO (PARA WORKERS) ===
# ==============================================================================

def process_user_video(user_item, drive_folder_path, watermark_path, watermark_width, target_gif_size_mb, safety_factor, max_clip_seconds_to_analyze, min_gif_duration, gif_fps, gif_width, secondary_api_url_template, log_file_path):
    """Processes video for a single user, creates a GIF, and returns status."""
    username = user_item.get('username')
    if not username:
        return username, "Skipped: No username found"

    print(f"\n--- Iniciando processamento (Worker): {username} ---")

    # Define caminhos para arquivos temporários e finais
    # Usar um identificador único por worker ou processo pode ser útil em ambientes complexos
    # Mas para este caso, o nome do usuário deve ser suficiente para arquivos temporários locais
    temp_video_path = f"temp_{username}_video.mp4"
    temp_palette_path = f"temp_{username}_paleta.png"
    temp_sample_gif_path = f"temp_{username}_sample.gif"
    final_gif_path = os.path.join(drive_folder_path, f"{username}.gif") # Caminho final no Drive

    # --- Registrar status 'Processando' no log ANTES de iniciar (dentro do worker) ---
    try:
        # Read the latest log state just before writing to avoid overwriting
        log_data = read_log(log_file_path)
        log_data[username] = "Processando"
        write_log(log_file_path, log_data)
        print(f"🔄 Status de '{username}' atualizado para 'Processando' no log pelo worker.")
    except Exception as e:
         print(f"⚠️ Atenção (Worker): Não foi possível atualizar log para 'Processando' para {username}: {e}")


    status = "Falha: Unknown error" # Default status
    try:
        # Etapa 4: Obter URL .m3u8
        m3u8_url = None
        preview = user_item.get('preview', {})
        src_url = preview.get('src', '')
        if src_url and src_url.endswith('.m3u8'):
            m3u8_url = src_url
        else:
            secondary_url = secondary_api_url_template.format(username=username)
            user_response = requests.get(secondary_url)
            user_response.raise_for_status()
            webrtc_info = user_response.json().get('liveInfo', {}).get('webRTC', {})
            m3u8_url = webrtc_info.get('cdnURL') or webrtc_info.get('edgeURL')

        if not m3u8_url:
            raise ValueError("Nenhuma URL .m3u8 válida encontrada.")
        print(f"🔗 URL .m3u8 encontrada para {username}.")

        # Etapa 5: Download do clipe de vídeo para análise
        print(f"🎬 Baixando clipe de {max_clip_seconds_to_analyze}s para análise para {username}...")
        # Using subprocess.run for better error handling
        result = subprocess.run(['ffmpeg', '-i', m3u8_url, '-t', str(max_clip_seconds_to_analyze), '-c', 'copy', '-bsf:a', 'aac_adtstoasc', temp_video_path, '-y'], capture_output=True, text=True)
        if result.returncode != 0:
           print(f"❌ FFmpeg Download Error para {username}:\n{result.stderr}")
           raise RuntimeError("Falha no comando FFmpeg de download.")

        if not os.path.exists(temp_video_path) or os.path.getsize(temp_video_path) == 0:
            raise ValueError("Falha ao baixar o vídeo de análise ou arquivo vazio.")

        # --- Etapa 6: LÓGICA DE CONTROLE DE TAMANHO ---
        print(f"📏 Iniciando processo de calibração de tamanho para {username}...")

        # 6a. Gerar paleta a partir do vídeo e da marca d'água
        print(f"🎨 Gerando paleta de cores com marca d'água para {username}...")
        # Using subprocess.run
        palette_cmd = f'ffmpeg -i "{temp_video_path}" -i "{watermark_path}" -filter_complex "[1:v] scale={watermark_width}:-1 [wm]; [0:v][wm] overlay=main_w-overlay_w-10:main_h-overlay_h-10,palettegen" "{temp_palette_path}" -y'
        result = subprocess.run(palette_cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ FFmpeg Palette Error para {username}:\n{result.stderr}")
            raise RuntimeError("Falha no comando FFmpeg palettegen.")

        # 6b. Criar um GIF de amostra de 1 segundo (sem marca d'água para calibração)
        print(f"📸 Criando GIF de amostra para calibração para {username}...")
        # Using subprocess.run
        sample_gif_cmd = f'ffmpeg -t 1 -i "{temp_video_path}" -i "{temp_palette_path}" -lavfi "fps={gif_fps},scale={gif_width}:-1:flags=lanczos [x]; [x][1:v] paletteuse" "{temp_sample_gif_path}" -y'
        result = subprocess.run(sample_gif_cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ FFmpeg Sample GIF Error para {username}:\n{result.stderr}")
            raise RuntimeError("Falha no comando FFmpeg sample gif.")

        if not os.path.exists(temp_sample_gif_path) or os.path.getsize(temp_sample_gif_path) == 0:
            raise ValueError("Falha ao criar o GIF de amostra para calibração ou arquivo vazio.")

        # 6c. Calcular a "taxa de dados" e a duração ideal
        sample_size_mb = os.path.getsize(temp_sample_gif_path) / (1024 * 1024)
        megabytes_per_second = sample_size_mb / 1.0

        if megabytes_per_second == 0:
            raise ValueError("Amostra com tamanho zero, impossível calcular a duração.")

        target_size_with_safety = target_gif_size_mb * safety_factor
        estimated_duration = target_size_with_safety / megabytes_per_second
        final_duration = min(estimated_duration, max_clip_seconds_to_analyze)

        print(f"📈 Amostra de 1s para {username} tem {sample_size_mb:.2f} MB. Taxa estimada: {megabytes_per_second:.2f} MB/s.")
        print(f"⏱️ Duração calculada para atingir {target_size_with_safety:.2f}MB para {username}: {final_duration:.2f} segundos.")

        if final_duration < min_gif_duration:
            raise ValueError(f"Duração calculada ({final_duration:.2f}s) é menor que o mínimo ({min_gif_duration}s).")


        # 6d. Criar o GIF final COM MARCA D'ÁGUA com a duração calculada
        print(f"✨ Criando GIF final com {final_duration:.2f} segundos e marca d'água para {username}...")
        # The filter complex includes scaling and overlaying the watermark
        # [0:v] is the video, [1:v] is the palette, [2:v] is the watermark
        # scale=WATERMARK_WIDTH:-1 resizes the watermark keeping aspect ratio
        # overlay=main_w-overlay_w-10:main_h-overlay_h-10 positions in bottom-right with 10px padding
        # Using subprocess.run
        final_gif_cmd = f'ffmpeg -t {final_duration} -i "{temp_video_path}" -i "{temp_palette_path}" -i "{watermark_path}" -lavfi "[0:v] fps={gif_fps},scale={gif_width}:-1:flags=lanczos [x]; [2:v] scale={watermark_width}:-1 [wm]; [x][wm] overlay=main_w-overlay_w-10:main_h-overlay_h-10 [y]; [y][1:v] paletteuse" "{final_gif_path}" -y'
        result = subprocess.run(final_gif_cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ FFmpeg Final GIF Error para {username}:\n{result.stderr}")
            raise RuntimeError("Falha no comando FFmpeg final gif.")

        # Etapa 7: Verificar sucesso
        if not os.path.exists(final_gif_path) or os.path.getsize(final_gif_path) == 0:
             raise ValueError("Falha ao criar o GIF final com marca d'água ou arquivo vazio.")

        final_gif_size_mb = os.path.getsize(final_gif_path) / (1024 * 1024)
        print(f"✅ Sucesso! GIF final criado com {final_gif_size_mb:.2f} MB para {username}.")

        status = "Processado"

    except Exception as e:
        print(f"❌ Ocorreu um erro ao processar '{username}': {e}.")
        status = f"Falha: {e}"

    finally:
        # Etapa 8: Limpar TODOS os arquivos temporários no worker
        for f in [temp_video_path, temp_palette_path, temp_sample_gif_path]:
            if os.path.exists(f):
                try:
                    os.remove(f)
                    # print(f"🗑️ Arquivo temporário removido (Worker): {f}") # Pode ser muito verboso
                except OSError as e:
                    print(f"⚠️ Atenção (Worker): Não foi possível remover o arquivo temporário {f}: {e}")

        # Retorna o resultado para o processo principal atualizar o log
        return username, status

# ==============================================================================
# === SCRIPT DE AUTOMAÇÃO PRINCIPAL ===
# ==============================================================================

print("Iniciando o processo de automação com controle de tamanho de arquivo, marca d'água e log de status...")

# --- Etapa 0: Verificar arquivo da marca d'água ---
if not os.path.exists(WATERMARK_PATH):
    print(f"❌ ERRO CRÍTICO: Arquivo da marca d'água não encontrado em '{WATERMARK_PATH}'.")
    print("Por favor, verifique se o caminho na variável WATERMARK_PATH está correto e se o Google Drive está montado.")
    raise FileNotFoundError(f"Arquivo da marca d'água não encontrado: {WATERMARK_PATH}")


# --- Etapa 1: Preparar Ambiente e Ler Registro ---
os.makedirs(DRIVE_FOLDER_PATH, exist_ok=True)
caminho_log = os.path.join(DRIVE_FOLDER_PATH, LOG_FILE_NAME)
log_data = read_log(caminho_log)

# Include 'Falha' in the set of users to re-process
processed_or_processing_users = {
    user for user, status in log_data.items()
    if status in ["Processando", "Processado"]
}

print(f"📖 Arquivo de registro lido. {len(log_data)} usuários registrados no total.")
print(f"⏳ {len([user for user, status in log_data.items() if status == 'Processando'])} usuários em processamento.")
print(f"✅ {len([user for user, status in log_data.items() if status == 'Processado'])} usuários já processados.")
print(f"❌ {len([user for user, status in log_data.items() if status.startswith('Falha')])} usuários com falha registrados.")


# --- Etapa 2: Obter e Filtrar Lista de Usuários ---
try:
    print(f"\nBuscando lista de usuários na API...")
    response = requests.get(PRIMARY_API_URL)
    response.raise_for_status()
    all_items = response.json().get('broadcasts', {}).get('items', [])
    if not all_items:
        print("❌ A API não retornou nenhum usuário.")
        users_to_process = [] # Initialize as empty list
    else:
        # Filter users who are NOT in the state 'Processando' or 'Processado' in the log
        # This will now include users with status 'Falha' for re-processing
        current_log_data = read_log(caminho_log)
        processed_or_processing_users = {
            user for user, status in current_log_data.items()
            if status in ["Processando", "Processado"]
        }
        users_to_process = [
            item for item in all_items
            if item.get('username') and item.get('username') not in processed_or_processing_users
        ]
        print(f"✅ API retornou {len(all_items)} usuários. Priorizando {len(users_to_process)} novos ou pendentes usuários para processamento paralelo.")
        if not users_to_process:
            print("🎉 Nenhum usuário novo ou pendente para processar nesta execução.")

except Exception as e:
    print(f"❌ Falha crítica na API: {e}")
    users_to_process = [] # Initialize as empty list in case of API failure

# --- Etapa 3: Processamento Paralelo com Pool ---
if users_to_process:
    print(f"\nIniciando processamento paralelo para {len(users_to_process)} usuários...")

    # Prepare arguments for the worker function
    # Use partial to fix arguments that are the same for all calls
    worker_func = partial(
        process_user_video,
        drive_folder_path=DRIVE_FOLDER_PATH,
        watermark_path=WATERMARK_PATH,
        watermark_width=WATERMARK_WIDTH,
        target_gif_size_mb=TARGET_GIF_SIZE_MB,
        safety_factor=SAFETY_FACTOR,
        max_clip_seconds_to_analyze=MAX_CLIP_SECONDS_TO_ANALYZE,
        min_gif_duration=MIN_GIF_DURATION,
        gif_fps=GIF_FPS,
        gif_width=GIF_WIDTH,
        secondary_api_url_template=SECONDARY_API_URL_TEMPLATE,
        log_file_path=caminho_log # Pass log file path to worker
    )

    # Remove the bulk "Processando" update here
    # The status will be updated by each worker individually

    processed_count = 0
    failed_count = 0

    try:
        # Create a pool of worker processes
        with multiprocessing.Pool(processes=NUM_WORKERS) as pool:
            # Use imap_unordered to process results as they complete (good for varied task times)
            # The worker_func returns (username, status)
            for username, status in pool.imap_unordered(worker_func, users_to_process):
                # Update log data in the main process (synchronously)
                # This update is still necessary to get the final status (Processado or Falha)
                # from the worker function and write it to the main log file.
                current_log_data = read_log(caminho_log) # Read latest state before updating
                current_log_data[username] = status
                write_log(caminho_log, current_log_data)
                print(f"✅/❌ Processamento concluído para '{username}' com status: {status}. Log atualizado pelo processo principal.")

                if status == "Processado":
                    processed_count += 1
                elif status.startswith("Falha"):
                    failed_count += 1

        print("\n\n🎉 Processamento paralelo concluído! 🎉")
        print(f"Usuários processados com sucesso nesta execução: {processed_count}")
        print(f"Usuários com falha no processamento nesta execução: {failed_count}")

    except Exception as e:
        print(f"\n❌ Ocorreu um erro durante a execução do pool de processos: {e}")
        # Attempt to update log for users still marked as "Processando" if possible
        print("⚠️ Tentando atualizar status de usuários que estavam em 'Processando'...")
        current_log_data = read_log(caminho_log)
        for item in users_to_process:
            username = item.get('username')
            # Only update if the status is still "Processando" (meaning it didn't finish or fail gracefully)
            if username and current_log_data.get(username) == "Processando":
                 current_log_data[username] = f"Falha: Interrompido ({e})"
                 print(f"❌ Status de '{username}' atualizado para 'Falha: Interrompido'.")
        write_log(caminho_log, current_log_data)


else:
    print("\n\n✨ Nenhuma tarefa de processamento paralela iniciada pois não há usuários novos ou pendentes. ✨")